## Phase Kickback

See details of motivation in [qiskit texbook](https://qiskit.org/textbook/ch-algorithms/grover.html). It shows the idea, how to convert "Classical Function" $f$ into a function of a desired type. The mechanism we have there is called *Phase Kickback*.

Phase gate.

$P_\phi = \begin{pmatrix} 1 & 0 \\ 0 & e^{i\phi}\end{pmatrix}$

vs global phase:

$GP_\phi = \begin{pmatrix} e^{i\phi} & 0 \\ 0 & e^{i\phi}\end{pmatrix}$

We already know, that a global phase may appear, when we apply a gate to an eigenstate:

$U|\psi_e\rangle= GP_\phi|\psi_e\rangle$.

What is we apply $CU=I\oplus U$ gate to $|\psi_k\rangle$ which is an eigenvector (eigenstate) of $U$?

$I\oplus U|+\psi_e\rangle= I\oplus GP_\phi|+\psi_e\rangle = P_\phi\otimes I|+\psi_e\rangle$

where:

$I\oplus GP_\phi = \begin{pmatrix} 1 & 0 & 0 & 0 \\ 0 & 1 & 0 & 0 \\ 0 & 0 & e^{i\phi} & 0 \\ 0 & 0 & 0 & e^{i\phi}\end{pmatrix}$

**Kickback**: Controlling qubit (register) accepts the phase change of $U$ eigenvalue (eigenphase).

What are eigenstates and eigenvalues of $X$?
Use [documentation](https://numpy.org/doc/stable/reference/generated/numpy.linalg.eig.html) if needed.

In [45]:
import numpy as np
X = np.array([[0., 1], [1, 0]])

## TODO compute eigenstates on X. How?

What does this mean?

Substitute $e, v$.

$X|e_1\rangle = v_1|e_1\rangle$

$X|e_2\rangle = v_2|e_2\rangle$


Let's check phase kickback for $CNOT$.

In [ ]:
from qiskit import QuantumRegister, ClassicalRegister, QuantumCircuit, Aer, execute
from qiskit.visualization import plot_histogram
import matplotlib.pyplot as plt

qr = QuantumRegister(2, "q")
cr = ClassicalRegister(1, "c")
qc = QuantumCircuit(qr, cr)

# we keep a controlling qubit in |+> to observe phase change.
# If phase does not change: control is:
# |0> =H=> |+> =H=> |0>,
# otherwise 
# |0> =H=> |+> =KB=> |-> =H=> |1>
qc.h(qr[0])
qc.barrier()

# TODO here: prepare and eigenstate of X in qr[1]


qc.barrier()

# Controlled NOT
qc.cx(0, 1)
qc.barrier()


# deciding whether control is in |+> or |->
qc.h(qr[0])

qc.measure(qr[0], cr[0])
print(qc.draw())

job = execute(qc, Aer.get_backend('qasm_simulator'), shots=100)
counts = job.result().get_counts(qc)
plot_histogram(counts)

In [ ]:
from qiskit.circuit.library import RYGate
from qiskit.quantum_info.operators import Operator
from math import pi

my_gate = RYGate(pi / 3)
my_operator = Operator(my_gate)

print(my_operator.to_matrix().real)

print("eigenvalues:", np.linalg.eig(my_operator)[0].round(3))
print("eigenstate-0:", np.linalg.eig(my_operator)[1][:, 0].round(3))
print("eigenstate-1:", np.linalg.eig(my_operator)[1][:, 1].round(3))

In [ ]:
qc = QuantumCircuit(2)
qc.append(my_gate, [1])
qc.append(my_gate.control(1), [0, 1])
qc.draw()

In [ ]:
# TODO let us repeat the whole experiment for an arbitrary gate: